In [1]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

/tmp/ipykernel_1713576/2441880800.py:2: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


In [2]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os
import muon
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=200)
                         
import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/dominik.klein/mambaforge/envs/

['default', 'despine']


/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [3]:
output_dir = "/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/OT_encodrine_analysis"

In [4]:
tp0 = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/OT_encodrine_analysis/1415_TemporalProblem.pkl")

In [5]:
tp1 = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/OT_encodrine_analysis/1516_TemporalProblem.pkl")

In [6]:
mudata = muon.read("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/mudata_with_annotation_all.h5mu")

In [7]:
endocrine_celltypes = [
    "Ngn3 low",
    "Ngn3 high",
    "Ngn3 high cycling",
    "Fev+",
    "Fev+ Alpha",
    "Fev+ Beta",
    "Fev+ Delta",
    "Eps. progenitors",
    "Alpha",
    "Beta",
    "Delta",
    "Epsilon"
]

In [8]:
mudata = mudata[mudata.obs["cell_type"].isin(endocrine_celltypes)].copy()

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [9]:
adata = mudata["rna"]
adata.obsm["X_MultiVI"] = mudata.obsm["X_MultiVI"]
adata.obsm["X_umap"] = mudata.obsm["X_umap"]
adata.obs["cell_type_refined"] = mudata.obs["cell_type_refined"]

In [10]:
def adapt_time(x):
    if x["stage"]=="E14.5":
        return 14.5
    if x["stage"]=="E15.5":
        return 15.5
    if x["stage"]=="E16.5":
        return 16.5
    raise ValueError

adata.obs['time'] = adata.obs.apply(adapt_time, axis=1).astype("category")

In [11]:
tp = TemporalProblem(adata)
tp = tp.prepare("time", joint_attr="X_MultiVI")

INFO     Ordering Index(['AAACAGCCAACAGCCT-1', 'AAACAGCCAACCCTCC-1', 'AAACAGCCACTAAGCC-1',                         
                'AAACAGCCATAAAGCA-1', 'AAACATGCAAGGTAAC-1', 'AAACATGCAAGGTGGC-1',                                  
                'AAACATGCAGCATTAT-1', 'AAACATGCATTGTGGC-1', 'AAACCAACAACCTAAT-1',                                  
                'AAACCAACATAGGCGA-1',                                                                              
                ...                                                                                                
                'TTTGAGTCACGTTACA-4', 'TTTGCGACATGCTATG-4', 'TTTGCGGAGACTCGCA-4',                                  
                'TTTGGCTGTGTGAGAG-4', 'TTTGGTAAGTAACGAG-4', 'TTTGGTGCACGTAATT-4',                                  
                'TTTGTGAAGCGGATTT-4', 'TTTGTGAAGTTGTCAA-4', 'TTTGTGTTCATTCATC-4',                                  
                'TTTGTTGGTCACAGCG-4'],                                  

/home/icb/dominik.klein/git_repos/moscot/src/moscot/problems/time/_mixins.py:986: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not (is_categorical_dtype(col) and is_numeric_dtype(col.cat.categories)):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [12]:
tp[14.5, 15.5].set_solution(tp0[14.5, 15.5].solution)
tp[15.5, 16.5].set_solution(tp1[15.5, 16.5].solution)

BirthDeathProblem[stage='solved', shape=(5185, 1699)]

In [13]:
tp.save(os.path.join(output_dir, "TemporalProblem.pkl"), overwrite=True)